# Convert a Depth Image to a Point Cloud

This example takes as an input a rrd with depth image data and uses
the Rerun DataFusion provided functions to create point cloud data.

In [ ]:
import rerun as rr
import rerun.utilities.datafusion.functions as rr_dfn
from datafusion import SessionContext
import pyarrow as pa

In [ ]:
# In this example, you will need a copy of the rrd from the `arkit_demo`.
# Update the file path below accordingly.

original_recording = rr.dataframe.load_recording("arkit_demo_data.rrd")

In [ ]:
# In order to turn this into a DataFusion DataFrame, we need to create a view
# that includes our `latest_at` specification. Otherwise the data for the Pinhole
# and the data for the DepthImage will be misaligned.

original_recording_view = (
    original_recording
    .view(index="log_time", contents="/world/camera_lowres/**", include_indicator_columns=True)
    .fill_latest_at()
)

In [ ]:
# Create the DataFusion context and DataFrame from record batches provided
# by the view above.

ctx = SessionContext()
df = ctx.create_dataframe([[r for r in original_recording_view.select()]])

In [ ]:
# This code section calls two functions. The first, `convert_depth_image_to_point_cloud`
# does the bulk of the calculation. The second copies over the transform components of
# the original camera data.

df_result = (
    df
    .transform(
        rr_dfn.convert_depth_image_to_point_cloud,
        "/world/camera_lowres/depth",
        "/world/camera_lowres",
        "/world/point_cloud"
    )
    .transform(
        rr_dfn.duplicate_components,
        "/world/camera_lowres",
        "/world/point_cloud",
        ["Translation3D", "RotationQuat", "Transform3DIndicator", "TransformRelation"]
    )
)

In [ ]:
# To visualize the results, create a recording stream

local_rec = rr.RecordingStream("depth_image_to_pt_cloud")
local_rec.spawn()

In [ ]:
# Send the original data so we can overlay on it

local_rec.send_recording(original_recording)

In [ ]:
# Convert the DataFusion DataFrame into a pyarrow Table and send it to the viewer

table_result = pa.table(df_result)
rr.dataframe.send_dataframe(table_result, rec=local_rec)